In [ ]:
import pandas as pd
import copy
import numpy as np

#### Load dataset

In [ ]:
df = pd.read_csv("data.csv")
print(df.shape)

### adding outcome variables and remove cases with missing outcome:  

In [ ]:
original_index = df.index

##### VAS change in pain score

In [ ]:
df_main = df
df_main.shape

In [ ]:
df = df.dropna(subset=['ptb_3764'])
df.shape

In [ ]:
def calculate_vas(df):
    dif = df.ptb_3764.values - df.pt3_3764.values
    new_df = copy.deepcopy(df)
    new_df['vasChaSco'] = dif
    return new_df

df = calculate_vas(df)
print(df.shape)

In [ ]:
df_main = calculate_vas(df_main)
df_main.shape

In [ ]:
print("miss value before calculate vasChasco: ", df.vasChaSco.isnull().sum())
print(df.shape)
df = df.dropna(subset=['vasChaSco'])
print("After removing miss value: ", df.shape)
print("miss value: ", df.vasChaSco.isnull().sum())

In [ ]:
df = df.drop(columns="pt3_3764")
df_main = df_main.drop(columns="pt3_3764")

In [ ]:
print("Shape of df:", df.shape)
print("Shape of df_main: ", df_main.shape)

##### KOOS qol change score change in pain score

In [ ]:
print("miss value at baseline: ", df.ptb_koos_qol_score.isnull().sum())
print("miss value after 3 months: ", df.pt3_koos_qol_score.isnull().sum())

In [ ]:
df = df.dropna(subset=['ptb_koos_qol_score'])
print(df.shape)

In [ ]:
def calculate_koos(df):
    dif = df.ptb_koos_qol_score.values - df.pt3_koos_qol_score.values
    new_df = copy.deepcopy(df)
    new_df['koosChaSco'] = dif
    return new_df

df = calculate_koos(df)
print(df.shape)

In [ ]:
df_main = calculate_koos(df_main)

In [ ]:
print("miss value after calculate KOOS: ", df.koosChaSco.isnull().sum())

In [ ]:
df = df.dropna(subset=['koosChaSco'])
print(df.shape)
print("miss value: ", df.koosChaSco.isnull().sum())

In [ ]:
df = df.drop(columns="pt3_koos_qol_score")
df_main = df_main.drop(columns="pt3_koos_qol_score")

In [ ]:
print("Shape of df:", df.shape)
print("Shape of df_main: ", df_main.shape)

##### 40m walking change score change in pain score

In [ ]:
print("miss value at baseline: ", df.testb_10391.isnull().sum())
print("miss value after 3 months: ", df.test3_10391.isnull().sum())

In [ ]:
df = df.dropna(subset=['testb_10391'])
print(df.shape)

In [ ]:
def calculate_speed_walk(df):
    dif = (40/df.testb_10391.values) - (40/df.test3_10391.values)
    new_df = copy.deepcopy(df)
    new_df['funcChaSco'] = dif
    return new_df

df = calculate_speed_walk(df)
print(df.shape)

In [ ]:
df_main = calculate_speed_walk(df_main)
print(df_main.shape)

In [ ]:
print("miss value after calculate speed walk: ", df.funcChaSco.isnull().sum())

In [ ]:
df = df.dropna(subset=['funcChaSco'])
print(df.shape)
print("miss value: ", df.funcChaSco.isnull().sum())

In [ ]:
print("vas:", df.vasChaSco.isna().sum())
print("koos:", df.koosChaSco.isna().sum())
print("fun:", df.funcChaSco.isna().sum())

In [ ]:
df = df.drop(columns=['test3_10391'])
df_main = df_main.drop(columns=['test3_10391'])

In [ ]:
print("Shape of df:", df.shape)
print("Shape of df_main: ", df_main.shape)

##### Exclude cases between 2016-05-13 & 2016-11-12 (because of duration of symptom)

In [ ]:
def filter_exclude_date_2016(df):
    filtered_df = df[(df['fysb_3629'].astype("datetime64[ns]") > '2016-05-13') & (df['fysb_3629'].astype("datetime64[ns]") < '2016-11-12')].index
    return df.drop(index = filtered_df)


df = filter_exclude_date_2016(df)
print(df.shape)
print(df_main.shape)

##### Excluded cases between 10th of April 2018 and 5th of August 2019 (because of pain area)

In [ ]:
def filter_exclude_date_2018(df):
    filtered_df = df[(df['ptb_3750'].astype("datetime64[ns]") >= '2018-04-10') & (df['ptb_3750'].astype("datetime64[ns]") <= '2019-08-05')].index
    return df.drop(index = filtered_df)


df = filter_exclude_date_2018(df)
print(df.shape)

#### VARIABLE PREPRATION

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### ptb_10226_b (Educational status)

In [ ]:
df.ptb_10226.value_counts(dropna=False)

In [ ]:
def calculate_ptb_10226_b(df):
    # Dichotomize data into "higher education than secondary education"
    df['ptb_10226_b'] = None  # Initialize with None to handle NaN values
    df.loc[df['ptb_10226'] == 3, 'ptb_10226_b'] = 1
    df.loc[df['ptb_10226'] == 4, 'ptb_10226_b'] = 1
    df.loc[df['ptb_10226'] == 5, 'ptb_10226_b'] = 1
    df.loc[df['ptb_10226'] == 1, 'ptb_10226_b'] = 0
    df.loc[df['ptb_10226'] == 2, 'ptb_10226_b'] = 0

    # Convert column 'ptb_10226_b' to logical type, keeping NaN as is
    df['ptb_10226_b'] = df['ptb_10226_b'].map({1: True, 0: False, None: None})

    return df

In [ ]:
df = calculate_ptb_10226_b(df)
df_main = calculate_ptb_10226_b(df_main)

In [ ]:
df.ptb_10226_b.value_counts(dropna=False)

In [ ]:
df_main.ptb_10226_b.value_counts(dropna=False)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### change NA to 0 in variables with only yes as option and Timing  

In [ ]:
df.ptb_15027_11.isna().sum()

In [ ]:
def change_NA_to_0_in_painarea(df):
    area_part_body_pain = ["ptb_15026_1",  "ptb_15026_2", "ptb_15026_3", "ptb_15026_4", "ptb_15026_5",
                 "ptb_15026_6", "ptb_15026_7", "ptb_15026_8", "ptb_15026_9", "ptb_15026_10",
                 "ptb_15026_11", "ptb_15026_12", "ptb_15026_13", "ptb_15026_14", "ptb_15026_15",
                 "ptb_15026_16", "ptb_15026_17", "ptb_15026_18", "ptb_15026_19", "ptb_15026_20",
                 "ptb_15026_21", "ptb_15026_22", "ptb_15026_23", "ptb_15026_24", "ptb_15026_25",
                 "ptb_15026_26", "ptb_15027_1",  "ptb_15027_2", "ptb_15027_3", "ptb_15027_4",
                 "ptb_15027_5", "ptb_15027_6", "ptb_15027_7", "ptb_15027_8", "ptb_15027_9",
                 "ptb_15027_10", "ptb_15027_11", "ptb_15027_12", "ptb_15027_13", "ptb_15027_14",
                 "ptb_15027_15", "ptb_15027_16", "ptb_15027_17", "ptb_15027_18", "ptb_15027_19",
                 "ptb_15027_20", "ptb_15027_21", "ptb_15027_22", "ptb_15027_23", "ptb_15027_24",
                 "ptb_15027_25", "ptb_15027_26", "ptb_15027_27", "ptb_15027_28", "ptb_15027_29",
                 "ptb_15027_30"]

    stichtag= pd.DataFrame(data={'a': ['2014-04-12']})
    stichtag = stichtag.a.astype('datetime64[ns]').values[0]
    survey_after_date = df[(df['ptb_3750'].notna()) & (df['ptb_3750'].astype('datetime64[ns]') >= stichtag)].index.values
    # survey_after_date = df['ptb_3750'].astype('datetime64') >= stichtag


    for ind, i in enumerate(area_part_body_pain):
        change_na = df[df[i].isnull()].index.values
        inter = list(set(survey_after_date).intersection(set(change_na)))
        # the intersection is always ZERO

        for j in inter:
            df.at[j, i] = 0

    return df

In [ ]:
df = change_NA_to_0_in_painarea(df)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### creating 2 new variables pain_area and ptb_low_back

In [ ]:
def create_pain_area(df):
    """
    Function to create the 'pain_area' column by summing specific columns, keeping NaN values.
    """
    columns_to_sum = [
        'ptb_15026_1', 'ptb_15026_2', 'ptb_15026_3', 'ptb_15026_4', 'ptb_15026_5',
        'ptb_15026_6', 'ptb_15026_7', 'ptb_15026_8', 'ptb_15026_9', 'ptb_15026_10',
        'ptb_15026_11', 'ptb_15026_12', 'ptb_15026_13', 'ptb_15026_14', 'ptb_15026_15',
        'ptb_15026_16', 'ptb_15026_17', 'ptb_15026_18', 'ptb_15026_19', 'ptb_15026_20',
        'ptb_15026_21', 'ptb_15026_22', 'ptb_15026_23', 'ptb_15026_24', 'ptb_15026_25',
        'ptb_15026_26', 'ptb_15027_1', 'ptb_15027_2', 'ptb_15027_3', 'ptb_15027_4',
        'ptb_15027_5', 'ptb_15027_6', 'ptb_15027_7', 'ptb_15027_8', 'ptb_15027_9',
        'ptb_15027_10', 'ptb_15027_11', 'ptb_15027_12', 'ptb_15027_13', 'ptb_15027_14',
        'ptb_15027_15', 'ptb_15027_16', 'ptb_15027_17', 'ptb_15027_18', 'ptb_15027_19',
        'ptb_15027_20', 'ptb_15027_21', 'ptb_15027_22', 'ptb_15027_23', 'ptb_15027_24',
        'ptb_15027_25', 'ptb_15027_26', 'ptb_15027_27', 'ptb_15027_28', 'ptb_15027_29',
        'ptb_15027_30'
    ]

    # Sum while propagating NaN values
    df['pain_area'] = df[columns_to_sum].sum(axis=1, skipna=False)
    return df


In [ ]:
def create_ptb_low_back(df):
    """
    Function to create the 'ptb_low_back' column based on conditions, preserving NaN values.
    """
  
    df['ptb_low_back'] = np.nan

    # Update rows where the condition is True (1)
    df.loc[
        (df['ptb_15027_13'] == 1) | (df['ptb_15027_11'] == 1) | 
        (df['ptb_15027_14'] == 1) | (df['ptb_15027_21'] == 1) | 
        (df['ptb_15027_22'] == 1) | (df['ptb_15027_12'] == 1), 
        'ptb_low_back'
    ] = 1

    # Update rows where the condition is False (0)
    df.loc[
        (df['ptb_15027_13'] == 0) & (df['ptb_15027_11'] == 0) &
        (df['ptb_15027_14'] == 0) & (df['ptb_15027_21'] == 0) &
        (df['ptb_15027_22'] == 0) & (df['ptb_15027_12'] == 0),
        'ptb_low_back'
    ] = 0


    df['ptb_low_back'] = df['ptb_low_back'].astype('boolean')
    
    return df

In [ ]:
df = create_pain_area(df)
df = create_ptb_low_back(df)
df_main = create_pain_area(df_main)
df_main = create_ptb_low_back(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

#####  X_ray signs, persons who did not get an x-ray ->

In [ ]:
def x_ray(df):
    x_ray_ind = df[df['fysb_3637'] == 1].index.values
    df.loc[x_ray_ind, 'fysb_9349'] = 3
    df = df.drop(columns=['fysb_3637'])
    return df

df = x_ray(df)
df_main = x_ray(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### changing chair-stand-test outcome to 0 for those who were not able to do one chair stand

In [ ]:
def chair_stand(df):
    chair_stand_ind = df[df['testb_10393'] == 2].index.values
    df.loc[chair_stand_ind, 'testb_10281'] = 0
    df = df.drop(columns=['testb_10393'])
    return df
df = chair_stand(df)
df_main = chair_stand(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### removing individual questions

In [ ]:
def remove_individual_question_variable(df, variables_dropped):
    df = df.drop(variables_dropped, axis=1)
    return df


variables_removed = ["ptb_15026_1",  "ptb_15026_2", "ptb_15026_3", "ptb_15026_4", "ptb_15026_5",
                 "ptb_15026_6", "ptb_15026_7", "ptb_15026_8", "ptb_15026_9", "ptb_15026_10",
                 "ptb_15026_11", "ptb_15026_12", "ptb_15026_13", "ptb_15026_14", "ptb_15026_15",
                 "ptb_15026_16", "ptb_15026_17", "ptb_15026_18", "ptb_15026_19", "ptb_15026_20",
                 "ptb_15026_21", "ptb_15026_22", "ptb_15026_23", "ptb_15026_24", "ptb_15026_25",
                 "ptb_15026_26", "ptb_15027_1",  "ptb_15027_2", "ptb_15027_3", "ptb_15027_4",
                 "ptb_15027_5", "ptb_15027_6", "ptb_15027_7", "ptb_15027_8", "ptb_15027_9",
                 "ptb_15027_10", "ptb_15027_11", "ptb_15027_12", "ptb_15027_13", "ptb_15027_14",
                 "ptb_15027_15", "ptb_15027_16", "ptb_15027_17", "ptb_15027_18", "ptb_15027_19",
                 "ptb_15027_20", "ptb_15027_21", "ptb_15027_22", "ptb_15027_23", "ptb_15027_24",
                 "ptb_15027_25", "ptb_15027_26", "ptb_15027_27", "ptb_15027_28", "ptb_15027_29",
                 "ptb_15027_30"]

df = remove_individual_question_variable(df, variables_removed)
df_main = remove_individual_question_variable(df_main, variables_removed)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### creating variables per right/left knee are additional joints affected

In [ ]:
# creating variables per right/left knee are addiational joints affected
def calculate_ptb_1028_5_6(df):

    df['ptb_10285_n'] = None
    df['ptb_10286_n'] = None

    # Process left side (ptb_10285_n)
    df.loc[df['ptb_10285_2'] == 1, 'ptb_10285_n'] = 0  # No additional joints affected
    df.loc[(df['ptb_10285_3'] == 1) | (df['ptb_10285_4'] == 1) | (df['ptb_10285_5'] == 1), 'ptb_10285_n'] = 1  # Additional joints affected

    # Process right side (ptb_10286_n)
    df.loc[df['ptb_10286_2'] == 1, 'ptb_10286_n'] = 0  # No additional joints affected
    df.loc[(df['ptb_10286_3'] == 1) | (df['ptb_10286_4'] == 1) | (df['ptb_10286_5'] == 1),'ptb_10286_n'] = 1  # Additional joints affected

    return df

df = calculate_ptb_1028_5_6(df)
df_main = calculate_ptb_1028_5_6(df_main)


In [ ]:
print(df.ptb_10285_n.value_counts())
print("Nan value:", df.ptb_10285_n.isna().sum())

In [ ]:
print("ptb_10285 nan value: ", df.ptb_10285_n.isna().sum())
print("ptb_10286 nan value: ", df.ptb_10286_n.isna().sum())

In [ ]:
print(df.ptb_10286_n.value_counts())
print("Nan value:", df.ptb_10286_n.isna().sum())

##### ptb_3754_b

In [ ]:
# creating a final variable about additional effected joints
def calculate_ptb_3754_b(df):
    df['ptb_3754_b'] = np.where(df['ptb_10285_n'].notna(), df['ptb_10285_n'], df['ptb_10286_n'])
    # 0 means no additonnal joint is affected and 1 means there are more than one joint affected
    # change to do you have additional effected joints? 1= yes 0 = no
    # change to boolean
    df['ptb_3754_b'] = df['ptb_3754_b'].astype(bool)
    return df

df = calculate_ptb_3754_b(df)
df_main = calculate_ptb_3754_b(df_main)

In [ ]:
def remove_irrelevant_variable(df):
    columns_to_remove = ['ptb_10285_n', 'ptb_10286_n', 'ptb_3754', 'ptb_10286_2', 'ptb_10286_3', 'ptb_10286_4',
                         'ptb_10286_5', 'ptb_10285_2', 'ptb_10285_3', 'ptb_10285_4', 'ptb_10285_5']
    df = df.drop(columns_to_remove, axis=1)
    return df

df = remove_irrelevant_variable(df)
df_main = remove_irrelevant_variable(df_main)

##### variable ptb_3792 Frequency of pain until exaustin making binary cut after once a week

In [ ]:
def calculate_ptb_3792_b(df):
    category_type_ptb_3792 = df['ptb_3792'].astype('category')

    df.loc[df['ptb_3792'] == 1, 'ptb_3792_b'] = 0
    df.loc[df['ptb_3792'] == 2, 'ptb_3792_b'] = 0
    df.loc[df['ptb_3792'] == 3, 'ptb_3792_b'] = 0
    df.loc[df['ptb_3792'] == 4, 'ptb_3792_b'] = 0
    df.loc[df['ptb_3792'] == 5, 'ptb_3792_b'] = 1
    df.loc[df['ptb_3792'] == 6, 'ptb_3792_b'] = 1
    df.loc[df['ptb_3792'] == 7, 'ptb_3792_b'] = 1

    df['ptb_3792_b'] = df['ptb_3792_b'].astype('bool')
    df = df.drop('ptb_3792', axis=1)

    return df

In [ ]:
df = calculate_ptb_3792_b(df)
df_main = calculate_ptb_3792_b(df_main)

##### variable ptb_3777 regarding the working status

In [ ]:
def calculate_ptb_3777_b(df):
    category_type_ptb_3777_b = df['ptb_3777'].astype('category')

    # dichotomize variable in are you working or studying yes(1)
    df.loc[df['ptb_3777'] == 1, 'ptb_3777_b'] = 1
    df.loc[df['ptb_3777'] == 2, 'ptb_3777_b'] = 0
    df.loc[df['ptb_3777'] == 3, 'ptb_3777_b'] = 0
    df.loc[df['ptb_3777'] == 4, 'ptb_3777_b'] = 0
    df.loc[df['ptb_3777'] == 5, 'ptb_3777_b'] = 0
    df.loc[df['ptb_3777'] == 6, 'ptb_3777_b'] = 0
    df.loc[df['ptb_3777'] == 7, 'ptb_3777_b'] = 0


    df['ptb_3777_b'] = df['ptb_3777_b'].astype(bool)
    df = df.drop('ptb_3777', axis=1)
    return df

In [ ]:
df = calculate_ptb_3777_b(df)
df_main = calculate_ptb_3777_b(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### ptb_koospain

In [ ]:
# !!!! **** we dont have ptb_3761. alternative is ptb_koospain_1  ****
def calculate_ptb_koospain_1(df):
    category_type_ptb_koospain = df['ptb_koospain_1'].astype('category')

    # dichotomize data in do you have pain every day? yes(1)
    df.loc[df['ptb_koospain_1'] == 0, 'ptb_koospain'] = 0
    df.loc[df['ptb_koospain_1'] == 1, 'ptb_koospain'] = 0
    df.loc[df['ptb_koospain_1'] == 2, 'ptb_koospain'] = 0
    df.loc[df['ptb_koospain_1'] == 3, 'ptb_koospain'] = 1
    df.loc[df['ptb_koospain_1'] == 4, 'ptb_koospain'] = 1

    df['ptb_koospain'] = df['ptb_koospain'].astype(bool)
    df = df.drop('ptb_koospain_1', axis=1)

    return df

In [ ]:
df = calculate_ptb_koospain_1(df)
df_main = calculate_ptb_koospain_1(df_main)

##### sick-leave - have you been on sick leave -> delete information on how long have on sick listed / recode 2 to 0 as no

In [ ]:
def sick_leave_ptb_3772(df):
    df.loc[df['ptb_3772'] == 2, 'ptb_3772'] = 0
    df.drop('ptb_3776', axis=1, inplace=True)
    return df

df = sick_leave_ptb_3772(df)
df.shape

In [ ]:
df_main = sick_leave_ptb_3772(df_main)
df_main.shape

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

#### Changing types of variables

##### change 2 to 0 in binary variables

In [ ]:
def get_value_counts(df, columns):
    return {col: df[col].value_counts() for col in columns}

columns = [
    'fysb_3639', 'fysb_3642', 'fysb_op', 'ptb_3772', 'ptb_11836',
    'testb_10392', 'ptb_10223', 'ptb_10224', 'ptb_10225', 'ptb_13118',
    'ptb_15553', 'ptb_15554', 'ptb_15555', 'ptb_15556', 'ptb_15557',
    'ptb_15558', 'ptb_15559', 'ptb_15560', 'ptb_15561', 'ptb_15562',
    'ptb_15563', 'ptb_15564', 'ptb_16316', 'ptb_3758', 'ptb_3760',
    'ptb_3762', 'ptb_3765'
]

value_counts = get_value_counts(df, columns)

for col, counts in value_counts.items():
    print(f"Value counts for {col}:\n{counts}\n")

In [ ]:
def change_type_variable(df):
    binary_variables_need_change = ['fysb_3639', 'fysb_3642', 'fysb_op', 'ptb_3772', 'ptb_11836',
                                    'testb_10392', 'ptb_10223', 'ptb_10224', 'ptb_10225', 'ptb_13118',
                                    'ptb_15553', 'ptb_15554', 'ptb_15555', 'ptb_15556', 'ptb_15557',
                                    'ptb_15558', 'ptb_15559', 'ptb_15560', 'ptb_15561', 'ptb_15562',
                                    'ptb_15563', 'ptb_15564', 'ptb_16316', 'ptb_3758', 'ptb_3760',
                                    'ptb_3762', 'ptb_3765']


    for i in binary_variables_need_change:
        ind = df[df[i] == 2].index.values
        for ii in ind:
            df.at[ii, i] = 0
    return df

df = change_type_variable(df)
df_main = change_type_variable(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### change NA to 0 in variables with only yes as option and no timig

In [ ]:
def change_na_to_0(df):
    change_variable_NA_0 = ['fysb_medicin']

    for i in change_variable_NA_0:
        for ii in df[df[i].isnull()].index.values:
            df.at[ii, i] = 0
    return df

df = change_na_to_0(df)
df_main = change_na_to_0(df_main)


In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

 ##### Change type of variables accordingly #############

In [ ]:
### Saved as a Integer/Nummeric variable ###
def change_integerNummeric (df):
    c = ['age', 'ptb_3764', 'testb_10281', 'ptb_4145', "ptb_eq5d_vas"]
    for i in c:
        df[i] = pd.to_numeric(df[i])
    return df

In [ ]:
df = change_integerNummeric (df)
df_main = change_integerNummeric (df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### Save and change as a NUMMERIC variable  ###

In [ ]:
def change_variables_to_nummeric(df):
    NumVar = ["fysb_BMI", "ptb_koos_qol_score", "ptb_eq5d_score_5l",
              "testb_10391", "ptb_ases_pain_score", "ptb_ases_other_score", "ptb_sf12_pcs",
              "ptb_sf12_mcs", "vasChaSco"]
    for i in NumVar:
        df[i] = pd.to_numeric(df[i])
    return df

In [ ]:
df = change_variables_to_nummeric(df)
df_main = change_variables_to_nummeric(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### Save and change as a FACTOR/categroies variable  ###

In [ ]:
def change_variable_to_categorical(df):
    df["fysb_9349"] = df["fysb_9349"].astype(str).astype("category")
    return df

df = change_variable_to_categorical(df)
df_main = change_variable_to_categorical(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### Save and change as a BINARY/LOGICAL variable ###

In [ ]:
def change_variables_to_binary_logical(df):
    df['ptb_32260'] = df['ptb_32260'].map({1: True, 2: False})
    logVar = ['fysb_3639', 'testb_10392', 'ptb_13118', 'ptb_15553', 'ptb_15554', "ptb_15555",
    "ptb_15556", "ptb_15557", "ptb_15558", "ptb_15559", "ptb_15560",
    "ptb_15561", "ptb_15562", "ptb_15563", "ptb_15564", "ptb_16316", "ptb_3758", "ptb_3760",
    "ptb_3762", "ptb_3765", "ptb_32260", "ptb_10223", "ptb_10224", "ptb_10225", "ptb_3772",
    "ptb_11836", "fysb_3642", "fysb_medicin", "fysb_op" ]
            #10225: 1 Living alone 2 Living with others (partner, family, friends or others) ==> are you alone: 1 = Yes = True
            # gender: are you male?: 1=yes =True, 2=No=False
    
    for i in logVar:    
        df[i] = df[i].astype('boolean')
        

    return df

In [ ]:
df = change_variables_to_binary_logical(df)
df_main = change_variables_to_binary_logical(df_main)

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### Gender

In [ ]:
df.gender.value_counts()

In [ ]:
df['gender'] = df['gender'].replace(2, 0)
df.gender.value_counts()

In [ ]:
df_main['gender'] = df_main['gender'].replace(2, 0)
df_main.gender.value_counts()

0: Female
1: Male

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### Checking missing value in Outcome variables:

In [ ]:
outcome_variable = ["vasChaSco", "koosChaSco", "funcChaSco"]

missing_values = df[outcome_variable].isnull().sum().sum()

print(f"There are {missing_values} missing values in the outcome variables.")


##### Handling 'nan' in fysb_9349

In [ ]:
nan_string_count = (df['fysb_9349'] == 'nan').sum()
print(f"Number of 'nan' string values: {nan_string_count}")

In [ ]:
df['fysb_9349'] = df['fysb_9349'].replace('nan', np.nan)

In [ ]:
nan_string_count = (df['fysb_9349'] == 'nan').sum()
print(f"Number of 'nan' string values: {nan_string_count}")

In [ ]:
df.fysb_9349.isna().sum()

#### list of ALL variables ()

In [ ]:
df.shape

In [ ]:
# Group 1: All variables still collected after 2023 (excluding information on diseases) 
group1 = ["ptb_11836", "fysb_3639", "ptb_3764", "fysb_symptomvarighed", "ptb_eq5d_score_5l", "testb_10391", "fysb_BMI", "age", 
          "ptb_eq5d_vas", "pain_area", "testb_10281", "ptb_koos_qol_score", "ptb_4145", "ptb_3765", "ptb_10226_b", "ptb_16316", 
          "ptb_koospain", "fysb_9349", "fysb_3642", "ptb_low_back", "ptb_3792_b", "gender", "fysb_op", "ptb_10225", "ptb_3758", 
          "ptb_3762", "ptb_13118", "fysb_medicin", "ptb_3777_b", "ptb_3772", "ptb_10223", "testb_10392", "ptb_10224", "ptb_3754_b", 
          "vasChaSco", "koosChaSco", "funcChaSco"]
print("group1:", len(group1))


# Group 2: All Green variables with 3 koos
group2 = ["ptb_11836", "fysb_3639", "ptb_3764", "fysb_symptomvarighed", "ptb_eq5d_score_5l", "testb_10391", "fysb_BMI", "age", "ptb_eq5d_vas", 
          "pain_area", "testb_10281", "ptb_koos_qol_score", "ptb_4145", "ptb_3765", "ptb_10226_b", "ptb_16316", "ptb_koospain", "fysb_9349", 
          "fysb_3642", "ptb_low_back", "ptb_3792_b", "gender", "fysb_op", "ptb_10225", "ptb_3758", "ptb_3762", "ptb_13118", "fysb_medicin", 
          "ptb_3777_b", "ptb_3772", "ptb_10223", "testb_10392", "ptb_10224", "ptb_3754_b","fysb_24607", 
          "fysb_24643", "fysb_24644", "fysb_24608", "fysb_24645", "fysb_24646", "fysb_24649", "fysb_24650", "fysb_24651",
          "ptb_koos_pain_score", "ptb_koos_func_score", "ptb_koos12_score",
          "vasChaSco", "koosChaSco", "funcChaSco"]
print("group2:", len(group2))

##### Checking mising value:

In [ ]:
df_group1 = df[group1]
print("shape group1:", df_group1.shape)

df_group2 = df[group2]
print("shape group2:", df_group2.shape)

In [ ]:
print("Miss value Group 1", df_group1.isna().sum().sum())
print("Miss value Group 2", df_group2.isna().sum().sum())

In [ ]:
df_cleaned_g1 = df_group1.dropna()
print("Shape of df_cleaned_g1:", df_cleaned_g1.shape)

In [ ]:
df_cleaned_g2 = df_group2.dropna()
print("Shape of df_cleaned_g2:", df_cleaned_g2.shape)

In [ ]:
df_cleaned_g2.to_csv("Supplement.csv", index=False)

In [ ]:
df = df_cleaned_g1

In [ ]:
print("df: ", df.shape)
print("df_main: ", df_main.shape)

##### Create df_main with common variables in df_included

In [ ]:
main_variable = ['age', 'gender', 'fysb_BMI', 'fysb_3639', 'fysb_symptomvarighed', 'fysb_9349', 'fysb_3642', 
'fysb_op', 'fysb_medicin', 'testb_10391', 'testb_10392', 'testb_10281', 'ptb_10223', 'ptb_10224', 
'ptb_10225', 'ptb_13118', 'ptb_16316', 'ptb_3758', 'ptb_3762', 'ptb_3764', 'ptb_3765', 'ptb_koos_qol_score', 
'ptb_eq5d_score_5l', 'ptb_eq5d_vas', 'ptb_3772', 'ptb_11836', 'ptb_4145', 'pain_area', 
'ptb_10226_b', 'ptb_koospain', 'ptb_low_back', 'ptb_3792_b', 'ptb_3777_b', 'ptb_3754_b', 'vasChaSco']

In [ ]:
df_main = df_main[main_variable]
df_included = df[main_variable]

In [ ]:
print("shape main", df_main.shape)
print("shape included", df_included.shape)

In [ ]:
df_ex = df_main[~df_main.index.isin(df.index)]

inc = df[['fysb_symptomvarighed']]
total= df_main[['fysb_symptomvarighed']]
ex = df_ex[['fysb_symptomvarighed']]

print("NaN value in total: : ", total.isna().sum())
print("NaN value in included: ", inc.isna().sum())
print("NaN value in excluded: : ", ex.isna().sum())

In [ ]:
df_ex.isna().sum()

In [ ]:
print("shape main", df_main.shape)
print("shape included", df_included.shape)
print("shape excluded", df_ex.shape)

##### Statistic

In [ ]:
df.shape

In [ ]:
df_ex.shape

In [ ]:
continues_variable = ["age", "fysb_BMI", "fysb_symptomvarighed", "testb_10391", "testb_10281", "ptb_3764", "pain_area", 
                    "ptb_4145", "ptb_eq5d_vas", "ptb_koos_qol_score", "ptb_eq5d_score_5l", "vasChaSco"]


binary_category_variables = [ "gender", "fysb_3639", "fysb_9349", "fysb_3642", "fysb_medicin", 
                             "fysb_op", "testb_10392", "ptb_10223", "ptb_10224", "ptb_10225", 
                             "ptb_10226_b", "ptb_13118", "ptb_16316", "ptb_3754_b", "ptb_3758", 
                             "ptb_koospain", "ptb_3762", "ptb_low_back", "ptb_3765", 
                             "ptb_3777_b", "ptb_3772", "ptb_11836", "ptb_3792_b"]

##### Included

In [ ]:
numerical_meta = {}
for c_name in continues_variable:
    target_column =  df_included[c_name]
    numerical_meta[c_name] = {
        "mean":target_column.mean(),
        "std": target_column.std(),
        "min": target_column.min(), 
        "max": target_column.max(),
        "nan": (target_column.isnull()).sum()
        # "data-type": target_column.dtype,
        # "median": target_column.median(),
       
     }
for i, (k, v) in enumerate(numerical_meta.items(), 1):
    print(f"{i}. {k}: {v}")

In [ ]:
categorical_meta = {}
for c_name in binary_category_variables:
    target_column =  df_included[c_name]
    categorical_meta[c_name] = {
        "data-type": target_column.dtype,
        "True": (target_column == True).sum(),
        "False": (target_column == False).sum(),
        "nan": (target_column.isnull()).sum()
        # "unique": np.unique(target_column.values, return_counts=True), 
        
     }
for i, (k, v) in enumerate(categorical_meta.items(), 1):
    print(f"{i}. {k}: {v}")

In [ ]:
print(df.fysb_9349.unique())
print("_______")
print(df_included.fysb_9349.unique())
print("_______")
print(df_included.fysb_9349.value_counts())
print("_______")
print(df_included.fysb_9349.isna().sum())

##### Excluded

In [ ]:
df_ex.shape

In [ ]:
numerical_meta = {}
for c_name in continues_variable:
    target_column =  df_ex[c_name]
    numerical_meta[c_name] = {
        "mean":target_column.mean(),
        "std": target_column.std(),
        "min": target_column.min(), 
        "max": target_column.max(),
        "nan": (target_column.isnull()).sum()
        # "data-type": target_column.dtype,
        # "median": target_column.median(),
       
     }
for i, (k, v) in enumerate(numerical_meta.items(), 1):
    print(f"{i}. {k}: {v}")

In [ ]:
categorical_meta = {}
for c_name in binary_category_variables:
    target_column =  df_ex[c_name]
    categorical_meta[c_name] = {
        # "data-type": target_column.dtype,
        "True": (target_column == True).sum(),
        "False": (target_column == False).sum(),
        "nan": (target_column.isnull()).sum()
        # "unique": np.unique(target_column.values, return_counts=True), 
        
     }
for i, (k, v) in enumerate(categorical_meta.items(), 1):
    print(f"{i}. {k}: {v}")

In [ ]:
print(df.fysb_9349.unique())
print("_______")
print(df_ex.fysb_9349.unique())
print("_______")
print(df_ex.fysb_9349.value_counts())
print("_______")
print(df_ex.fysb_9349.isna().sum())